In [107]:
import numpy as np
import matplotlib.pyplot as plt
import sys
print("imports successful")

grid_count_crust = 640 # would need 637.1 for 20 km spacing
#1275 for 5 km spacing
coordinates = np.linspace(-6371, 6371, grid_count_crust)
grid_1d_size_crust = coordinates[1]-coordinates[0]

# Generate the grid coordinates using meshgrid
x_coordinates, y_coordinates, z_coordinates = np.meshgrid(coordinates, coordinates, coordinates)

# Calculate the distance of each point from the origin using vectorized operations
distances_squared = x_coordinates**2 + y_coordinates**2 + z_coordinates**2

# Find indices where distance is less than or equal to the radius squared
crust_indices = np.logical_and(6350**2 < distances_squared, distances_squared <= 6371**2)

# Extract valid coordinates using boolean indexing
crust_grid = np.stack((x_coordinates[crust_indices], y_coordinates[crust_indices], z_coordinates[crust_indices]), axis=-1)

print("crust grid created successfully")

imports successful
crust grid created successfully


In [108]:
grid_count_mantle = 250 # would need 637.1 for 20 km spacing
#320 for 40 km spacing
#285 for 45 km spacing
#215 for 60km spacing
#160 for 80km spacing
coordinates = np.linspace(-6371, 6371, grid_count_mantle)
grid_1d_size_mantle = coordinates[1]-coordinates[0]

# Generate the grid coordinates using meshgrid
x_coordinates, y_coordinates, z_coordinates = np.meshgrid(coordinates, coordinates, coordinates)

# Calculate the distance of each point from the origin using vectorized operations
distances_squared = x_coordinates**2 + y_coordinates**2 + z_coordinates**2
CLM_indices = np.logical_and(6196**2 < distances_squared, distances_squared <= 6350**2)
CLM_grid = np.stack((x_coordinates[CLM_indices], y_coordinates[CLM_indices], z_coordinates[CLM_indices]), axis=-1)

DM_indices1 = np.logical_and(4202**2 < distances_squared, distances_squared <= 5200**2)
DM_grid1 = np.stack((x_coordinates[DM_indices1], y_coordinates[DM_indices1], z_coordinates[DM_indices1]), axis=-1)

DM_indices2 = np.logical_and(5200**2 < distances_squared, distances_squared <= 6196**2)
DM_grid2 = np.stack((x_coordinates[DM_indices2], y_coordinates[DM_indices2], z_coordinates[DM_indices2]), axis=-1)

EM_indices = np.logical_and(3480**2 < distances_squared, distances_squared <= 4202**2)
EM_grid = np.stack((x_coordinates[EM_indices], y_coordinates[EM_indices], z_coordinates[EM_indices]), axis=-1)

print("mantle grid created successfully")

mantle grid created successfully


In [109]:
print("setting densities and abundances")

# Placeholder definitions for IDE recognition
A_Th_c = A_U_c = A_Th_CLM = A_U_CLM = A_Th_DM1 = A_U_DM1 = A_Th_DM2 = A_U_DM2 = A_Th_EM = A_U_EM = 0

# Define the parameter sets
low_abundances = {
    "A_Th_c": 5 * (10**(-6)),
    "A_U_c": 1 * (10**(-6)),
    "A_Th_CLM": 53 * (10**(-9)),
    "A_U_CLM": 13 * (10**(-9)),
    "A_Th_DM": 17.52 * (10**(-9)),
    "A_U_DM": 6.4 * (10**(-9)),
    "A_Th_EM": 90 * (10**(-9)),
    "A_U_EM": 15 * (10**(-9))
}

mid_abundances = {
    "A_Th_c": 5 * (10**(-6)),
    "A_U_c": 1 * (10**(-6)),
    "A_Th_CLM": 147 * (10**(-9)),
    "A_U_CLM": 33 * (10**(-9)),
    "A_Th_DM": 21.9 * (10**(-9)),
    "A_U_DM": 8 * (10**(-9)),
    "A_Th_EM": 147 * (10**(-9)),
    "A_U_EM": 33 * (10**(-9))
}

high_abundances = {
    "A_Th_c": 5 * (10**(-6)),
    "A_U_c": 1 * (10**(-6)),
    "A_Th_CLM": 427 * (10**(-9)),
    "A_U_CLM": 83 * (10**(-9)),
    "A_Th_DM": 26.28 * (10**(-9)),
    "A_U_DM": 9.6 * (10**(-9)),
    "A_Th_EM": 221 * (10**(-9)),
    "A_U_EM": 57 * (10**(-9))
}

# Function to select the parameter set
def select_abundances(abundance_set):
    globals().update(abundance_set)

# Choose the desired parameter set by name
abundances = low_abundances  # Change to mid_abundances or high_abundances as needed
select_abundances(abundances)

print(A_Th_c)


rho_c = 2.7
rho_CLM = 3.3 #g/cm^3 probably
rho_DM = 3.3
rho_EM = 3.3
print("densities and abundances set successfully")

#we have pretty much everything to compute things separately!
# compute for crust
#set position of SNO+

print("setting detector position")

SNO_r = np.array([0, 0, 6369])

setting densities and abundances
4.9999999999999996e-06
densities and abundances set successfully
setting detector position


In [110]:
def calc_relative_dist(points_array):
    # Calculate the Euclidean distance using vectorized operations
    relative_distances = np.linalg.norm(points_array - SNO_r, axis=1)
    return relative_distances

#define Delta function for an array of points and energies
#inputs: appropriate deltam _ij^2, energy array [MeV], points array
#relative distance calculated in km, hence Delta scaled up by a factor of 1000 to account for m-km conversion

def Delta_ij(energy_array, points_array, delta_m_ij_squared):
    # Calculate relative distances
    relative_distance_array = calc_relative_dist(points_array)
    
    # Reshape energy_array to perform element-wise division
    energy_array_reshaped = energy_array.reshape(-1, 1)
    
    # Calculate Delta using vectorized operations
    Delta = (1.27 * delta_m_ij_squared * relative_distance_array * 1000) / (energy_array_reshaped)
    
    return Delta

In [111]:
def P_ee_full_optimized(energy_array, points_array, theta_12, theta_13, delta_m_21_squared):
    
    #P_ee = np.empty((len(energy_array), len(points_array))) # np.empty more efficient
    #than np.zeros or whatever
    #might not need this at all though
    relative_distance_array = calc_relative_dist(points_array)
    
    Delta_31 = Delta_ij(energy_array, points_array, delta_m_31_squared)
    print("Delta_31 computed successfully")
    Delta_12 = Delta_ij(energy_array, points_array, delta_m_21_squared)
    print("Delta_12 computed successfully")

    A = (np.cos(theta_13))**4 * (np.sin(2 * theta_12))**2
    B = np.sin(2 * theta_13)**2

    sin_squared_Delta_31 = np.sin(Delta_31) ** 2
    sin_squared_Delta_12 = np.sin(Delta_12) ** 2
    
    print("values computed successfully")

    P_ee = 1 - (A * sin_squared_Delta_12 + B * sin_squared_Delta_31)
    print("P_ee computed successfully")
    
    return P_ee

In [112]:
#set standard oscillation parameters
print("setting standard oscillation parameters")

theta_12 = 0.5903 #rad
theta_23 = 0.8430 #rad
theta_13 = 0.1503 #rad

delta_m_21_squared = 7.39 * 10**(-5) #eV^2
delta_m_32_squared = 2.449 * 10**(-3) #eV^2
delta_m_31_squared = delta_m_32_squared
print("defining energy array")
energy_array = np.linspace(1.8, 3.3, 100)

setting standard oscillation parameters
defining energy array


In [113]:
def sigma_IBD(energy_array):
    # optimized with numpy
    
    m_e = 0.511 #MeV
    m_p = 938 #MeV
    m_n = 941 #MeV
    E_th = 1.806 #MeV
    
    sigma = ((energy_array - E_th + m_e)**2) * ((1 - (m_e)**2/((energy_array - E_th + m_e)**2))**(1/2))
    
    return sigma
#all these + sources in Earth model more bins -3 (for example)

In [114]:
print("setting lambda and mu values")

lambda_U = 4.916
lambda_Th = 1.563
mu_U = 235
mu_Th = 232

setting lambda and mu values


In [115]:

#get fluxes
print("getting emission fluxes")
energy_array_U = []
dn_dE_U = []

with open('U238_spectrum.txt', 'r') as file:
    for line in file:
        # Split each line into columns
        columns = line.split()

        # Convert the elements to float and append to arrays
        energy_array_U.append(float(columns[0]))
        dn_dE_U.append(float(columns[1]))

# Scale down all energies by a factor of 1000
energy_array_U = np.array(energy_array_U) / 1000
dn_dE_U = np.array(dn_dE_U)

# plt.plot(energy_array_U, dn_dE_U)
# plt.xlabel('E_nu [MeV]')
# plt.yscale('log')
# plt.ylabel('Intensity (some units, doesn\'t matter)')
# plt.title('Energy spectrum of geonus from U238 decay')
# 
# # Add shaded region between 1.8 MeV and 3.2 MeV
# plt.axvspan(1.8, 3.2, alpha=0.3, color='gray')
# # Enable minor ticks on x-axis
# plt.minorticks_on()
# plt.show()

print("done for Uranium, moving on to Thorium")

energy_array_Th = []
dn_dE_Th = []
with open('Th232_spectrum.txt', 'r') as file:
    for line in file:
        # Split each line into columns
        columns = line.split()

        # Convert the elements to float and append to arrays
        energy_array_Th.append(float(columns[0]))
        dn_dE_Th.append(float(columns[1]))

# Scale down all energies by a factor of 1000
energy_array_Th = np.array(energy_array_Th) / 1000
dn_dE_Th = np.array(dn_dE_Th)

# plt.plot(energy_array_Th, dn_dE_Th)
# plt.xlabel('E_nu [MeV]')
# plt.yscale('log')
# plt.ylabel('Intensity (some units, doesn\'t matter)')
# plt.title('Energy spectrum of geonus from Th232 decay')
# # Add shaded region between 1.8 MeV and 3.2 MeV
# plt.axvspan(1.8, 3.2, alpha=0.3, color='gray')
# # Enable minor ticks on x-axis
# plt.minorticks_on()
# plt.show()
# Plot U238 decay data (blue line)
# plt.plot(energy_array_U, dn_dE_U, label='U238 decays', color='blue')
# 
# # Plot Th232 decay data (red line)
# plt.plot(energy_array_Th, dn_dE_Th, label='Th232 decays', color='red')
# 
# plt.xlabel('E_nu [MeV]')
# plt.yscale('log')
# plt.ylabel('Intensity (some units, doesn\'t matter)')
# plt.title('Energy spectrum of geonus')
# 
# # Add shaded region between 1.8 MeV and 3.3 MeV
# plt.axvspan(1.8, 3.3, alpha=0.3, color='gray')
# 
# # Enable minor ticks on x-axis
# plt.minorticks_on()
# 
# plt.legend(loc='upper right')
# 
# plt.show()
def rebin_counts(initial_bins, counts_in_initial_bins, final_bin_midpoints):
    """
    Rebins counts data.

    Parameters:
    - initial_bins: array, bin edges of the initial data
    - counts_in_initial_bins: array, counts in each initial bin
    - final_bin_midpoints: array, midpoints of the final desired bins

    Returns:
    - counts_in_final_bins: array, counts in each final bin
    """

    # Calculate bin midpoints of the initial bins
    bin_midpoints = (initial_bins[:-1] + initial_bins[1:]) / 2

    # Use np.histogram to calculate counts in final bins
    counts_in_final_bins, _ = np.histogram(initial_bins, bins=np.concatenate([initial_bins, [2 * initial_bins[-1] - initial_bins[-2]]]), weights=counts_in_initial_bins)

    # Interpolate the counts to the final bin midpoints
    counts_in_final_bins = np.interp(final_bin_midpoints, bin_midpoints, counts_in_final_bins[:-1])

    return counts_in_final_bins

print("rebin to match energy array")

dn_dE_rebinned_U = rebin_counts(energy_array_U, dn_dE_U, energy_array)
dn_dE_rebinned_Th = rebin_counts(energy_array_Th, dn_dE_Th, energy_array)

getting emission fluxes
done for Uranium, moving on to Thorium
rebin to match energy array


In [116]:
print("computing IBD cross section")
sigma = sigma_IBD(energy_array)

computing IBD cross section


/tmp/ipykernel_47249/3884541781.py:9: RuntimeWarning: invalid value encountered in sqrt
  sigma = ((energy_array - E_th + m_e)**2) * ((1 - (m_e)**2/((energy_array - E_th + m_e)**2))**(1/2))


In [117]:
#DEFINE ENERGY WEIGHTS

w_E_Th = ((lambda_Th)/(mu_Th)) * sigma * dn_dE_rebinned_Th
w_E_U = ((lambda_U)/(mu_U)) * sigma * dn_dE_rebinned_U

print("Shape of w_E_Th:", w_E_Th.shape)
print("Shape of w_E_U:", w_E_U.shape)

Shape of w_E_Th: (100,)
Shape of w_E_U: (100,)


In [118]:
#DEFINE FUNCTION TO COMPUTE POSITION WEIGHTS

def get_position_weights(points_array, theta_12, grid_1d_size, delta_m_21_squared, A_Th, A_U, rho):
    
    dV = grid_1d_size**3
    
    relative_distance_array = calc_relative_dist(points_array)
    print("Relative distance array computed successfully")
    
    position_weights_Th = (A_Th  * rho * dV) / (4 * np.pi * relative_distance_array**2)
    position_weights_U = (A_Th  * rho * dV) / (4 * np.pi * relative_distance_array**2)
    
    return position_weights_Th, position_weights_U
    

In [119]:
# CALCULATE ALL SURVIVAL PROBABILITIES
P_ee_c = P_ee_full_optimized(energy_array, crust_grid, theta_12, theta_13, delta_m_21_squared)
print("Shape of P_ee_c:", P_ee_c.shape)
print("crust done")

P_ee_CLM = P_ee_full_optimized(energy_array, CLM_grid, theta_12, theta_13, delta_m_21_squared)
print("Shape of P_ee_CLM:", P_ee_CLM.shape)
print("CLM done")

P_ee_DM1 = P_ee_full_optimized(energy_array, DM_grid1, theta_12, theta_13, delta_m_21_squared)
print("Shape of P_ee_DM1:", P_ee_DM1.shape)
print("DM1 done")

P_ee_DM2 = P_ee_full_optimized(energy_array, DM_grid2, theta_12, theta_13, delta_m_21_squared)
print("Shape of P_ee_DM2:", P_ee_DM2.shape)
print("DM2 done")

P_ee_EM = P_ee_full_optimized(energy_array, EM_grid, theta_12, theta_13, delta_m_21_squared)
print("Shape of P_ee_EM:", P_ee_EM.shape)
print("EM done")

Delta_31 computed successfully
Delta_12 computed successfully
values computed successfully
P_ee computed successfully
Shape of P_ee_c: (100, 1344768)
crust done
Delta_31 computed successfully
Delta_12 computed successfully
values computed successfully
P_ee computed successfully
Shape of P_ee_CLM: (100, 147128)
CLM done
Delta_31 computed successfully
Delta_12 computed successfully
values computed successfully
P_ee computed successfully
Shape of P_ee_DM1: (100, 540176)
DM1 done
Delta_31 computed successfully
Delta_12 computed successfully
values computed successfully
P_ee computed successfully
Shape of P_ee_DM2: (100, 791944)
DM2 done
Delta_31 computed successfully
Delta_12 computed successfully
values computed successfully
P_ee computed successfully
Shape of P_ee_EM: (100, 260560)
EM done


In [120]:
# CALCULATE POSITION WEIGHTS
# Calculate position weights for _c
w_pos_Th_c, w_pos_U_c = get_position_weights(crust_grid, theta_12, grid_1d_size_crust, delta_m_21_squared, A_Th_c, A_U_c, rho_c)
w_pos_Th_c = np.nan_to_num(w_pos_Th_c, nan=0)
w_pos_U_c = np.nan_to_num(w_pos_U_c, nan=0)
print("Shape of w_pos_Th_c:", w_pos_Th_c.shape)
print("Shape of w_pos_U_c:", w_pos_U_c.shape)
print("Position weights computed successfully for c")

# Calculate position weights for _CLM
w_pos_Th_CLM, w_pos_U_CLM = get_position_weights(CLM_grid, theta_12, grid_1d_size_mantle, delta_m_21_squared, A_Th_CLM, A_U_CLM, rho_CLM)
w_pos_Th_CLM = np.nan_to_num(w_pos_Th_CLM, nan=0)
w_pos_U_CLM = np.nan_to_num(w_pos_U_CLM, nan=0)
print("Shape of w_pos_Th_CLM:", w_pos_Th_CLM.shape)
print("Shape of w_pos_U_CLM:", w_pos_U_CLM.shape)
print("Position weights computed successfully for CLM")

# Calculate position weights for _DM1
w_pos_Th_DM1, w_pos_U_DM1 = get_position_weights(DM_grid1, theta_12, grid_1d_size_mantle, delta_m_21_squared, A_Th_DM1, A_U_DM1, rho_DM)
w_pos_Th_DM1 = np.nan_to_num(w_pos_Th_DM1, nan=0)
w_pos_U_DM1 = np.nan_to_num(w_pos_U_DM1, nan=0)
print("Shape of w_pos_Th_DM1:", w_pos_Th_DM1.shape)
print("Shape of w_pos_U_DM1:", w_pos_U_DM1.shape)
print("Position weights computed successfully for DM1")

# Calculate position weights for _DM2
w_pos_Th_DM2, w_pos_U_DM2 = get_position_weights(DM_grid2, theta_12, grid_1d_size_mantle, delta_m_21_squared, A_Th_DM2, A_U_DM2, rho_DM)
w_pos_Th_DM2 = np.nan_to_num(w_pos_Th_DM2, nan=0)
w_pos_U_DM2 = np.nan_to_num(w_pos_U_DM2, nan=0)
print("Shape of w_pos_Th_DM2:", w_pos_Th_DM2.shape)
print("Shape of w_pos_U_DM2:", w_pos_U_DM2.shape)
print("Position weights computed successfully for DM2")

# Calculate position weights for _EM
w_pos_Th_EM, w_pos_U_EM = get_position_weights(EM_grid, theta_12, grid_1d_size_mantle, delta_m_21_squared, A_Th_EM, A_U_EM, rho_EM)
w_pos_Th_EM = np.nan_to_num(w_pos_Th_EM, nan=0)
w_pos_U_EM = np.nan_to_num(w_pos_U_EM, nan=0)
print("Shape of w_pos_Th_EM:", w_pos_Th_EM.shape)
print("Shape of w_pos_U_EM:", w_pos_U_EM.shape)
print("Position weights computed successfully for EM")



Relative distance array computed successfully
Shape of w_pos_Th_c: (1344768,)
Shape of w_pos_U_c: (1344768,)
Position weights computed successfully for c
Relative distance array computed successfully
Shape of w_pos_Th_CLM: (147128,)
Shape of w_pos_U_CLM: (147128,)
Position weights computed successfully for CLM
Relative distance array computed successfully
Shape of w_pos_Th_DM1: (540176,)
Shape of w_pos_U_DM1: (540176,)
Position weights computed successfully for DM1
Relative distance array computed successfully
Shape of w_pos_Th_DM2: (791944,)
Shape of w_pos_U_DM2: (791944,)
Position weights computed successfully for DM2
Relative distance array computed successfully
Shape of w_pos_Th_EM: (260560,)
Shape of w_pos_U_EM: (260560,)
Position weights computed successfully for EM


In [121]:

#because we have big arrays, the whole thing needs to be computed in parts

In [122]:
numerator_Th_c = np.nansum(w_pos_Th_c[:, np.newaxis] * w_E_Th * P_ee_c.T)
numerator_U_c = np.nansum(w_pos_U_c[:, np.newaxis] * w_E_U * P_ee_c.T)
print("Numerators computed for c")

numerator_Th_CLM = np.nansum(w_pos_Th_CLM[:, np.newaxis] * w_E_Th * P_ee_CLM.T)
numerator_U_CLM = np.nansum(w_pos_U_CLM[:, np.newaxis] * w_E_U * P_ee_CLM.T)
print("Numerators computed for CLM")

numerator_Th_DM1 = np.nansum(w_pos_Th_DM1[:, np.newaxis] * w_E_Th * P_ee_DM1.T)
numerator_U_DM1 = np.nansum(w_pos_U_DM1[:, np.newaxis] * w_E_U * P_ee_DM1.T)
print("Numerators computed for DM1")

numerator_Th_DM2 = np.nansum(w_pos_Th_DM2[:, np.newaxis] * w_E_Th * P_ee_DM2.T)
numerator_U_DM2 = np.nansum(w_pos_U_DM2[:, np.newaxis] * w_E_U * P_ee_DM2.T)
print("Numerators computed for DM2")

numerator_Th_EM = np.nansum(w_pos_Th_EM[:, np.newaxis] * w_E_Th * P_ee_EM.T)
numerator_U_EM = np.nansum(w_pos_U_EM[:, np.newaxis] * w_E_U * P_ee_EM.T)
print("Numerators computed for EM")



Numerators computed for c
Numerators computed for CLM
Numerators computed for DM1
Numerators computed for DM2
Numerators computed for EM


In [123]:
denominator_Th_c = np.nansum(w_pos_Th_c[:, np.newaxis] * w_E_Th)
denominator_U_c = np.nansum(w_pos_U_c[:, np.newaxis] * w_E_U)
print("Denominators computed for c")

denominator_Th_CLM = np.nansum(w_pos_Th_CLM[:, np.newaxis] * w_E_Th)
denominator_U_CLM = np.nansum(w_pos_U_CLM[:, np.newaxis] * w_E_U)
print("Denominators computed for CLM")

denominator_Th_DM1 = np.nansum(w_pos_Th_DM1[:, np.newaxis] * w_E_Th)
denominator_U_DM1 = np.nansum(w_pos_U_DM1[:, np.newaxis] * w_E_U)
print("Denominators computed for DM1")

denominator_Th_DM2 = np.nansum(w_pos_Th_DM2[:, np.newaxis] * w_E_Th)
denominator_U_DM2 = np.nansum(w_pos_U_DM2[:, np.newaxis] * w_E_U)
print("Denominators computed for DM2")

denominator_Th_EM = np.nansum(w_pos_Th_EM[:, np.newaxis] * w_E_Th)
denominator_U_EM = np.nansum(w_pos_U_EM[:, np.newaxis] * w_E_U)
print("Denominators computed for EM")


Denominators computed for c
Denominators computed for CLM
Denominators computed for DM1
Denominators computed for DM2
Denominators computed for EM


In [124]:
#calculate average P_ee

P_ee_average = (numerator_Th_c + numerator_Th_CLM + numerator_Th_DM1 + numerator_Th_DM2 + numerator_Th_EM + numerator_U_c + numerator_U_CLM + numerator_U_DM1 + numerator_U_DM2 + numerator_U_EM) / (denominator_Th_c + denominator_Th_CLM + denominator_Th_DM1 + denominator_Th_DM2 + denominator_Th_EM + denominator_U_c + denominator_U_CLM + denominator_U_DM1 + denominator_U_DM2 + denominator_U_EM)

print("Average of P_ee:", P_ee_average)

Average of P_ee: 0.524537515107165


In [125]:
# now calculate stdev
# Calculate numerator for _c with adjustment by average
numerator_var_Th_c = np.nansum(w_pos_Th_c[:, np.newaxis] * w_E_Th * (P_ee_c.T - P_ee_average)**2)
numerator_var_U_c = np.nansum(w_pos_U_c[:, np.newaxis] * w_E_U * (P_ee_c.T - P_ee_average))
print("Numerators computed for c")

# Calculate numerator for _CLM with adjustment by average
numerator_var_Th_CLM = np.nansum(w_pos_Th_CLM[:, np.newaxis] * w_E_Th * (P_ee_CLM.T - P_ee_average)**2)
numerator_var_U_CLM = np.nansum(w_pos_U_CLM[:, np.newaxis] * w_E_U * (P_ee_CLM.T - P_ee_average))
print("Numerators computed for CLM")

# Calculate numerator for _DM1 with adjustment by average
numerator_var_Th_DM1 = np.nansum(w_pos_Th_DM1[:, np.newaxis] * w_E_Th * (P_ee_DM1.T - P_ee_average)**2)
numerator_var_U_DM1 = np.nansum(w_pos_U_DM1[:, np.newaxis] * w_E_U * (P_ee_DM1.T - P_ee_average))
print("Numerators computed for DM1")

# Calculate numerator for _DM2 with adjustment by average
numerator_var_Th_DM2 = np.nansum(w_pos_Th_DM2[:, np.newaxis] * w_E_Th * (P_ee_DM2.T - P_ee_average)**2)
numerator_var_U_DM2 = np.nansum(w_pos_U_DM2[:, np.newaxis] * w_E_U * (P_ee_DM2.T - P_ee_average))
print("Numerators computed for DM2")

# Calculate numerator for _EM with adjustment by average
numerator_var_Th_EM = np.nansum(w_pos_Th_EM[:, np.newaxis] * w_E_Th * (P_ee_EM.T - P_ee_average)**2)
numerator_var_U_EM = np.nansum(w_pos_U_EM[:, np.newaxis] * w_E_U * (P_ee_EM.T - P_ee_average))
print("Numerators computed for EM")

Numerators computed for c
Numerators computed for CLM
Numerators computed for DM1
Numerators computed for DM2
Numerators computed for EM


In [126]:
P_ee_variance = (numerator_var_Th_c + numerator_var_Th_CLM + numerator_var_Th_DM1 + numerator_var_Th_DM2 + numerator_var_Th_EM + numerator_var_U_c + numerator_var_U_CLM + numerator_var_U_DM1 + numerator_var_U_DM2 + numerator_var_U_EM) / (denominator_Th_c + denominator_Th_CLM + denominator_Th_DM1 + denominator_Th_DM2 + denominator_Th_EM + denominator_U_c + denominator_U_CLM + denominator_U_DM1 + denominator_U_DM2 + denominator_U_EM)

print("P_ee_variance:", P_ee_variance)

P_ee_variance: 0.005259669960816898


In [127]:
P_ee_stdev = np.sqrt(P_ee_variance)
print("Standard Deviation (stdev):", P_ee_stdev)

Standard Deviation (stdev): 0.07252358210138891
